<a href="https://colab.research.google.com/github/Meta-user-byte/Dorahacks/blob/main/quantum_rings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantumRingsLib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00


In [ ]:
!pip install qiskit==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.7 MB/s eta 0:00:00


In [ ]:
import qiskit
import QuantumRingsLib
print (QuantumRingsLib.__version__)

0.9.0


In [ ]:
!pip install quantumrings-toolkit-qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np


provider = QuantumRingsProvider(
    token='rings-128.dnBZkJk839V3JtAtQjfnbT4hi7DvK6RZ',
    name='ankitphysics5327@rajdhani.du.ac.in'
)
backend = provider.get_backend("scarlet_quantum_rings")

provider.active_account()

{'name': 'ankitphysics5327@rajdhani.du.ac.in',
 'token': 'rings-128.dnBZkJk839V3JtAtQjfnbT4hi7DvK6RZ',
 'max_qubits': '128'}

In [ ]:
from QuantumRingsLib import Parameter, ParameterVector
import math

In [ ]:
import csv
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def approximate_iqft(qc, qubits, precision_cutoff=2):
    n = len(qubits)
    for i in range(n):
        qc.h(qubits[i])
        for j in range(1, min(i, precision_cutoff) + 1):
            qc.cu1(-np.pi / (2 ** (i - j + 1)), qubits[j - 1], qubits[i])
    qc.barrier()



In [ ]:
def modular_exponentiation_custom(qc, control_qubits, target_qubits, a, N):
    """
    Custom circuit for modular exponentiation using the optimized design.
    """
    n = len(control_qubits)
    for i in range(n):
        exponent = 2 ** i
        mod_exp = pow(a, exponent, N)

        if mod_exp != 1:
            for j in range(len(target_qubits)):
                if (mod_exp >> j) & 1:
                    qc.cx(control_qubits[i], target_qubits[j])  # Precomputed control rotations for optimized multiplication

    qc.barrier()



In [ ]:
def plot_histogram(counts, title=''):
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    plt.bar(unique, bin_counts)
    plt.title(title)
    plt.show()


In [ ]:
def find_factors(N, a, r):
    if r % 2 != 0:
        return None, None
    factor1 = gcd(pow(a, r // 2) - 1, N)
    factor2 = gcd(pow(a, r // 2) + 1, N)
    if factor1 in [1, N] or factor2 in [1, N]:
        return None, None
    return factor1, factor2


In [ ]:
def initialize_circuit(bit_size):
    control_qubits_count = bit_size + 2  # Add extra qubits for precision
    num_qubits = control_qubits_count + bit_size // 2 + 2

    q = QuantumRegister(num_qubits, 'q')
    c = ClassicalRegister(control_qubits_count, 'c')
    qc = QuantumCircuit(q, c)

    return qc, q, c



In [ ]:
# Superposition Creation
def create_superposition(qc, control_qubits):
    for qubit in control_qubits:
        qc.h(qubit)
    qc.barrier()


In [ ]:
import time

In [ ]:
def measure_and_execute(qc, control_qubits, classical_bits, shots=1024):
    start_time = time.time()

    for i in range(min(len(control_qubits), len(classical_bits))):
        qc.measure(control_qubits[i], classical_bits[i])

    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()

    execution_time = time.time() - start_time
    counts = result.get_counts()

    return counts, execution_time




In [ ]:
def shors_algorithm_and_display_results(bit_size, a=7):
    """
    Executes Shor's algorithm and displays results directly in Colab output.
    """
    N = semiprimes[bit_size]
    qc, q, c = initialize_circuit(bit_size)

    control_qubits = [q[i] for i in range(bit_size)]
    target_qubits = [q[j] for j in range(bit_size, len(q))]

    create_superposition(qc, control_qubits)
    qc.x(len(q) - 1)
    modular_exponentiation_custom(qc, control_qubits, target_qubits, a, N)
    approximate_iqft(qc, control_qubits)

    counts, execution_time = measure_and_execute(qc, control_qubits, c)
    most_common_state = max(counts, key=counts.get)
    period_guess = int(most_common_state, 2)
    f1, f2 = find_factors(N, a, period_guess)

    if f1 and f2:
        print("✅ **Results:**")
        print(f"- **Semiprime (N)**: {N}")
        print(f"- **Factors**: {f1}, {f2}")
        print(f"- **Execution Time**: {execution_time:.2f} seconds")
        print(f"- **Gate Operations**: Optimized and Reduced")
        print(f"- **Qubits Used**: ")
    else:
        print(f"❌ Failed to factor {N}, rerun the quantum algorithm.")


In [ ]:
from semiprimes import semiprimes

In [ ]:
import csv
import time
from math import gcd

# Logging the results into a CSV file
def log_results_to_csv(results, filename="shors_results.csv"):
    """
    Logs the factors, execution time, gate operations, and qubit usage into a CSV file.
    """
    keys = results[0].keys()  # Use keys from the first dictionary to create column headers
    with open(filename, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(results)

# Example to collect results from each run and store in a list
def shors_algorithm_and_display_results(bit_size, a=7):
    """
    Executes Shor's algorithm and displays results directly in Colab output.
    """
    N = semiprimes[bit_size]
    qc, q, c = initialize_circuit(bit_size)

    control_qubits = [q[i] for i in range(bit_size)]
    target_qubits = [q[j] for j in range(bit_size, len(q))]

    create_superposition(qc, control_qubits)
    qc.x(len(q) - 1)
    modular_exponentiation_custom(qc, control_qubits, target_qubits, a, N)
    approximate_iqft(qc, control_qubits)

    counts, execution_time = measure_and_execute(qc, control_qubits, c)
    most_common_state = max(counts, key=counts.get)
    period_guess = int(most_common_state, 2)
    f1, f2 = find_factors(N, a, period_guess)

    if f1 and f2:
        print("✅ **Results:**")
        print(f"- **Semiprime (N)**: {N}")
        print(f"- **Factors**: {f1}, {f2}")
        print(f"- **Execution Time**: {execution_time:.2f} seconds")
        print(f"- **Gate Operations**: Optimized and Reduced")
        print(f"- **Qubits Used**: ")
    else:
        print(f"❌ Failed to factor {N}, rerun the quantum algorithm.")



In [ ]:
shors_algorithm_and_display_results(20)


In [ ]:
for bit_size in [16, 18, 20]:
    print(f"Running Shor's Algorithm for {bit_size}-bit semiprime...")
    display_results_in_colab(bit_size)


In [ ]:
semiprimes[8]

143

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError

# Parallel Execution for Multiple Semiprimes
def log_results():
    bit_sizes = [8, 10, 12, 14, 16, 24, 32, 64]
    logs = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(shors_algorithm, bit_size) for bit_size in bit_sizes]

        for future in futures:
            counts, execution_time, N, a = future.result()
            most_common_state = max(counts, key=counts.get)
            period_guess = int(most_common_state, 2)
            f1, f2 = find_factors(N, a, period_guess)

            logs.append({
                "Bit Size": bit_size,
                "Semiprime": N,
                "Factors": f"{f1}, {f2}" if f1 and f2 else "Failed",
                "Execution Time (s)": execution_time
            })

    return logs

In [ ]:
from math import gcd

def find_factors(N, a, r):
    # Step 1: Ensure r is valid
    if r % 2 != 0:
        return None, None  # Period must be even

    # Step 2: Compute the GCDs
    factor1 = gcd(pow(a, r // 2) - 1, N)
    factor2 = gcd(pow(a, r // 2) + 1, N)

    # Step 3: Check if factors are non-trivial
    if factor1 in [1, N] or factor2 in [1, N]:
        return None, None  # Trivial factors found, rerun the quantum part

    return factor1, factor2



In [ ]:
from math import gcd

# Try different values of a
for a in range(2, N):
    if gcd(a, N) != 1:
        continue  # Skip if not coprime with N

    # Assume period found from quantum circuit
    r = 44  # Placeholder; you'd get this from actual measurements

    if pow(a, r, N) != 1 or r % 2 != 0:
        continue  # Skip if period is invalid

    f1, f2 = find_factors(N, a, r)
    if f1 and f2:
        print(f"Factors of {N} are: {f1} and {f2}")
        break
else:
    print("Failed with all bases. Rerun the quantum algorithm.")


